In [1]:
import numpy as np
import pandas as pd

In [2]:
%pip install -U gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 12.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: gensim
    Found existing installation: gensim 4.0.1
    Uninstalling gensim-4.0.1:
      Successfully uninstalled gensim-4.0.1
Note: you may need to restart the kernel to use updated packages.


In [6]:
from nltk.tokenize import word_tokenize
import os
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [7]:
data = []

df=pd.read_csv('/kaggle/input/apna-squad/train_input_paragraph_25.csv')
df=df[df.theme=='Nigeria']
for i in range(df.shape[0]):
    rw=df.iloc[i]
    data.append(df['paragraph'])

data=data[0]
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]

In [8]:
# df['id']=list(i for i in range(df.shape[0]))
df.head()

,id,paragraph,theme
0,1,"NigeriaSat-2, Nigeria's second satellite, was ...",Nigeria
1,2,"Waste management including sewage treatment, t...",Nigeria
2,3,Nigeria's foreign policy was tested in the 197...,Nigeria
3,4,The Niger Delta Nembe Creek Oil field was disc...,Nigeria
4,5,The Kingdom of Nri of the Igbo people consolid...,Nigeria


In [9]:
#FULL
max_epochs = 1
vec_size = 728
alpha = 0.025

model_full = gensim.models.doc2vec.Doc2Vec(vector_size=vec_size, min_count=2, epochs=70,dm=1,alpha = 0.025)
  
model_full.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model_full.train(tagged_data,
                total_examples=model_full.corpus_count,
                epochs=model_full.epochs)
    # decrease the learning rate
    model_full.alpha -= 0.0002
    # fix the learning rate, no decay
    model_full.min_alpha = model_full.alpha

model_full.save("d2v.model")
print("Model Saved")

iteration 0
Model Saved


In [10]:
from gensim.models.doc2vec import Doc2Vec

model_full= Doc2Vec.load("d2v.model")
#to find the vector of a document which is not in training data
test_data = word_tokenize("I love chatbots".lower())
v1 = model_full.infer_vector(test_data)

In [11]:
#Pretrained
test_data_dir = os.path.join(gensim.__path__[0], 'test', 'test_data')
lee_train_file = os.path.join(test_data_dir, 'lee_background.cor')
lee_test_file = os.path.join(test_data_dir, 'lee.cor')
import smart_open

def read_corpus(fname, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

train_corpus = list(read_corpus(lee_train_file))
test_corpus = list(read_corpus(lee_test_file, tokens_only=True))
model_pre = gensim.models.doc2vec.Doc2Vec(vector_size=vec_size, min_count=2, epochs=40,dm=1)
model_pre.build_vocab(train_corpus)
model_pre.train(train_corpus, total_examples=model_pre.corpus_count, epochs=model_pre.epochs)



In [12]:
dict_vals={}
for i in range(df.shape[0]):
    dict_vals[df.iloc[i]['theme']]=[]
    
for i in range(df.shape[0]):
    dict_vals[df.iloc[i]['theme']].append(df.iloc[i]['paragraph'])
dict_models={}
for i in dict_vals.keys():
    data=dict_vals[i]
    tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]
    model_semi = gensim.models.doc2vec.Doc2Vec(vector_size=vec_size, min_count=2, epochs=70,dm=1,alpha = 0.025)
  
    model_semi.build_vocab(tagged_data)

    for epoch in range(max_epochs):
        print('iteration {0}'.format(epoch),"theme = ",i)
        model_semi.train(tagged_data,
                    total_examples=model_semi.corpus_count,
                    epochs=model_semi.epochs)
        # decrease the learning rate
        model_semi.alpha -= 0.0002
        # fix the learning rate, no decay
        model_semi.min_alpha = model_semi.alpha

    model_semi.save(i+"-d2v.model")
    dict_models[i]=model_semi
    print("Model Saved")
    

iteration 0 theme =  Nigeria
Model Saved


In [14]:
ground_truth=pd.read_csv('/kaggle/input/apna-squad/train_ground_truth_25.csv')
ground_truth.head()

,question_id,paragraph_id,answers
0,1,[1],['Abuja']
1,2,[2],['Kubwa Community']
2,3,[1],['NigeriaSat-2']
3,4,[3],['Organisation for African Unity']
4,5,[4],['decades']


In [15]:
ground_truth=pd.read_csv('/kaggle/input/apna-squad/train_ground_truth_25.csv')
ground_truth= ground_truth[:45]
df1=pd.read_csv('/kaggle/input/apna-squad/train_input_paragraph_25.csv')
df1=df1[df1.theme== 'Nigeria']
df1=df1.reset_index(drop=True)
df2=pd.read_csv('/kaggle/input/apna-squad/train_input_question_25.csv')
df2= df2[df2.theme== 'Nigeria']
df2= df2.reset_index(drop=True)
dict_new_df={'paragraph':[],'question':[],'question_id':[],'para_id':[],'answer':[],'theme':[]}
for i in range(ground_truth.shape[0]):
    rw=ground_truth.iloc[i]
    para_id=rw['paragraph_id'][1:-1]
    if not(len(para_id)==0):
        para_id=int(para_id)
        q_id=rw['question_id']
        them=df2.loc[df2['id']==q_id]['theme'].values[0]
        para=df1.loc[df1['id']==para_id]['paragraph'].values[0]
        ques=df2.loc[df2['id']==q_id]['question'].values[0]
        dict_new_df['paragraph'].append(para)
        dict_new_df['question'].append(ques)
        dict_new_df['para_id'].append(para_id)
        dict_new_df['question_id'].append(q_id)
        dict_new_df['theme'].append(them)
        dict_new_df['answer'].append(True)
        
df=pd.DataFrame(dict_new_df)
df.head()

,paragraph,question,question_id,para_id,answer,theme
0,"NigeriaSat-2, Nigeria's second satellite, was ...",Where is Nigeria's second satellite's ground r...,1,1,True,Nigeria
1,"Waste management including sewage treatment, t...",What Nigerian community has the worst unsustai...,2,2,True,Nigeria
2,"NigeriaSat-2, Nigeria's second satellite, was ...",What was Nigeria's second satellite called?,3,1,True,Nigeria
3,Nigeria's foreign policy was tested in the 197...,What group was Nigeria a founding member of?,4,3,True,Nigeria
4,The Niger Delta Nembe Creek Oil field was disc...,How long have international oil companies been...,5,4,True,Nigeria


In [16]:
from scipy import spatial
def get_thresh():
    thresh_pre,thresh_full=[0,0,0,0],[0,0,0,0]
    dict_thresh_semi={}
    for i in dict_vals.keys():
        dict_thresh_semi[i]=[0,0,0,0]
    
    for i in range(df.shape[0]):
        rw=df.iloc[i]
        if i%1000==0:
            print("At i = ",i)
        context=rw['paragraph']
        ques=rw['question']
        test_context = word_tokenize(context.lower())
        test_ques = word_tokenize(ques.lower())
        
        v1_full = model_full.infer_vector(test_context)
        v2_full = model_full.infer_vector(test_ques)
        
        v1_pre = model_pre.infer_vector(test_context)
        v2_pre = model_pre.infer_vector(test_ques)
        
        model_sm=dict_models[rw['theme']]
        
        v1_semi = model_sm.infer_vector(test_context)
        v2_semi = model_sm.infer_vector(test_ques)
        
        result_full = 1 - spatial.distance.cosine(v1_full, v2_full)
        result_pre = 1 - spatial.distance.cosine(v1_pre, v2_pre)
        result_semi = 1 - spatial.distance.cosine(v1_semi, v2_semi)
        
        if rw['answer']==True:
            thresh_pre[0]+=result_pre
            thresh_full[0]+=result_full
            dict_thresh_semi[rw['theme']][0]+=result_semi
            dict_thresh_semi[rw['theme']][2]+=1
            thresh_full[2]+=1
            thresh_pre[2]+=1
            
        else :
            thresh_pre[1]+=result_pre
            thresh_full[1]+=result_full
            dict_thresh_semi[rw['theme']][1]+=result_semi
            dict_thresh_semi[rw['theme']][3]+=1
            thresh_full[3]+=1
            thresh_pre[3]+=1
    
    return thresh_pre,thresh_full,dict_thresh_semi
    

In [17]:
r1,r2,r3=get_thresh()

At i =  0


In [18]:
thresh_pre=r1[0]/r1[2]
thresh_full=r2[0]/r2[2]
thresh_semi={}
for i in dict_vals.keys():
    try:
        thresh_semi[i]=r3[i][0]/r3[i][2]
    except:
        pass

print("Thresh for pretrained model correct context-question similarity = ",thresh_pre)
print("Thresh for full trained model correct context-question similarity = ",thresh_full)
print("Thresh for Nigeria: ",thresh_semi['Nigeria'])


Thresh for pretrained model correct context-question similarity =  0.30571457361802457
Thresh for full trained model correct context-question similarity =  0.7646287308798896
Thresh for Nigeria:  0.76415563027064


In [19]:

try:
    thresh_answer_full=r2[0]/r2[2]
    thresh_unanswer_full=r2[1]/r2[3]
    thresh_answer_pre=r1[0]/r1[2]
    thresh_unanswer_pre=r1[1]/r1[3]
except:
    pass
dict_thresh_semi={}
for i in dict_vals.keys():
        dict_thresh_semi[i]=[0,0]
for i in r3.keys():
    try:
        dict_thresh_semi[i][0]=r3[i][0]/r3[i][2]
        dict_thresh_semi[i][1]=r3[i][1]/r3[i][3]
    except:
        pass


In [20]:
dict_context_embed={}
for i in range(df.shape[0]):
    dict_context_embed[df.iloc[i]['para_id']]=[]
for i in range(df.shape[0]):
    rw=df.iloc[i]
    if (i)%1000==0:
        print("At i = ",i)
    context=rw['paragraph']
    test_context = word_tokenize(context.lower())
    v1_full = model_full.infer_vector(test_context)
    v1_pre = model_pre.infer_vector(test_context)
    model_sm=dict_models[rw['theme']]
    v1_semi = model_sm.infer_vector(test_context)
    dict_context_embed[df.iloc[i]['para_id']]=[v1_pre,v1_full,v1_semi]
    
    

At i =  0


In [22]:
sample_context=[]
sample_ques="None"
sample_theme="None"
sample_ids=[]
for i in range(5):
    rw=df.iloc[i]
    sample_context.append(rw['paragraph'])
    sample_ques=rw['question']
    sample_theme=rw['theme']
    sample_ids.append(rw['para_id'])
    

In [61]:

# encoded_inputs = tokenizer(
#     questions=["What is love ?","What is love ?","What is love ?"],
#     titles=["Haddaway","Haddaway","Haddaway"],
#     texts=["'What Is Love' is a song recorded by the artist Haddaway","'What Is Love' is a song recorded by the artist Haddaway","'What Is Love' is a song recorded by the artist Haddaway"],
#     return_tensors="tf",
# )
# outputs = model(encoded_inputs)
# start_logits = outputs.start_logits
# end_logits = outputs.end_logits
# relevance_logits = outputs.relevance_logits

In [62]:
# start_indices=np.argmax(start_logits.numpy(),axis=1).tolist()
# end_indices=np.argmax(end_logits.numpy(),axis=1).tolist()
# best_ans=np.argmax(relevance_logits.numpy())

In [63]:
# tokenizer.decode(encoded_inputs['input_ids'][best_ans][start_indices[best_ans]:end_indices[best_ans]+1])

In [133]:
from transformers import TFDPRReader, DPRReaderTokenizer

tokenizer = DPRReaderTokenizer.from_pretrained("facebook/dpr-reader-single-nq-base")
model = TFDPRReader.from_pretrained("facebook/dpr-reader-single-nq-base", from_pt=True)

Downloading:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RobertaTokenizer'. 
The class this function is called from is 'DPRReaderTokenizer'.


TypeError: stat: path should be string, bytes, os.PathLike or integer, not NoneType

In [139]:
%pip install nlu
%pip install pyspark==3.0.2

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.8/204.8 MB 4.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 7.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.0.2-py2.py3-none-any.whl size=205186690 sha256=d3f4770682e96a4e38f0833ce9ed23243b5329f52ac8390c188c252a47171e94
  Stored in directory: /root/.cache/pip/wheels/9a/39/f6/970565f38054a830e9a8593f388b36e14d75dba6c6fdafc1ec
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
Note: you may need to restart the kernel to use updated packages.


In [83]:
import tensorflow as tf
def get_reader_answer(ques,retriever_contexts,tm,method='dpr'):
    if method=='dpr':
        question=[]
        theme=[]
        for i in range(len(retriever_contexts)):
            question.append(ques)
            theme.append(tm)
        encoded_inputs = tokenizer(
            questions=question,
            titles=theme,
            texts=retriever_contexts,
            return_tensors="tf",
            padding=True,
            truncation=True
        )
        outputs = model(encoded_inputs)
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits
        relevance_logits = outputs.relevance_logits

        start_indices=np.argmax(start_logits.numpy(),axis=1).tolist()
        end_indices=np.argmax(end_logits.numpy(),axis=1).tolist()
        best_ans=np.argmax(relevance_logits.numpy())
        relevance_logits= tf.nn.softmax(relevance_logits)
        return tokenizer.decode(encoded_inputs['input_ids'][best_ans][start_indices[best_ans]:end_indices[best_ans]+1]),relevance_logits.numpy()[best_ans], retriever_contexts[best_ans]
    
    elif method=='roberta':
        return None

In [84]:
v1,v2, par=get_reader_answer(sample_ques,sample_context,sample_theme,'dpr')

In [85]:
from scipy import spatial

In [124]:
import time
def predict(context,questions,context_ids,model_type='full',theme=None,top_k=5,thresh=0.40):
    st=time.time()
    sim_scores=[]
    context_embed=0
    question_embed=0
    context_token = word_tokenize(context.lower())
    if model_type=='full':
        context_embed = model_full.infer_vector(context_token)
    elif model_type=='pre':
        context_embed = model_pre.infer_vector(context_token)
    else :
        model_sm=dict_models[theme]
        context_embed = model_sm.infer_vector(context_token)
    questions_embed=[]
    for i in range(len(questions)):
        result1=0
        if model_type=='full':
            question_embed = dict_context_embed[context_ids[i]][1]
            result1 = 1 - spatial.distance.cosine(context_embed, question_embed)
        elif model_type=='pre':
            question_embed = dict_context_embed[context_ids[i]][0]
            result1 = 1 - spatial.distance.cosine(context_embed, question_embed)
        else :
            question_embed = dict_context_embed[context_ids[i]][2]
            result1 = 1 - spatial.distance.cosine(context_embed, question_embed)    
        questions_embed.append(question_embed)
        sim_scores.append(result1)
    
    array = np.array(sim_scores)
    indices = np.argsort(array)[::-1]
    
#     print("Got similarities score as ",array)
    
    retriever_contexts=[]
    for i in range(min(top_k,len(indices))):
        if array[indices[i]]>=thresh:
            retriever_contexts.append(questions[indices[i]])
    
    print("Time for embedding comparison = ",time.time()-st)
    answer=""
    socre=1
    par=""
    if len(retriever_contexts)>0:
        answer,socre, par=get_reader_answer(context,retriever_contexts,theme,method='dpr')
    
    print("Time For Total prediction  = ",time.time()-st)
    return answer,socre,par
    
    
    
    

In [125]:
v1,v2,par=predict(sample_ques,sample_context,sample_ids,'full',sample_theme,1,0.6)

Time for embedding comparison =  0.009574413299560547
Time For Total prediction  =  0.5219631195068359


In [126]:
v1

'decades'

In [127]:
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
        return re.sub(regex, ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [128]:
# Allowed to make changes.
theme_df= pd.read_csv('/kaggle/input/apna-squad/train_input_paragraph_25.csv')
def pred_theme_ans(questions, pred_out):
    theme = questions[0]["theme"]
    reqtheme_df= theme_df[theme_df['theme']==theme].reset_index(drop=True)
    sample_context= reqtheme_df.paragraph.values.tolist()
    sample_ids= reqtheme_df.id.values.tolist()
    for step,question in enumerate(questions):

        ans = {}
        ans["question_id"] = question["id"]
        ques= normalize_answer(question['question'])

        ans['paragraph_id']= -1
        ans['answers']= " "
        an, scor, paragra= predict(question['question'],sample_context,sample_ids,'full',theme,2, 0.6)
        #change threshold here
        threshold=0.65
        if(scor>=threshold):
            ans['answers']= an
            context= paragra
            try:
                ans['paragraph_id']= reqtheme_df[theme_df['paragraph']==context].id.values[0]
            except:
                ans['paragraph_id']=-1

        pred_out.append(ans)

In [129]:
# path= "/kaggle/input/apnasquad/"
# # All theme prediction.
# questions = json.loads(pd.read_csv(path+"train_input_question_25.csv").to_json(orient="records"))
# theme_intervals = json.loads(pd.read_csv(path+"train_theme_interval_25.csv").to_json(orient="records"))
# pred_out = []
# theme_inf_time = {}
# for theme_interval in theme_intervals:
#     theme_ques = questions[int(theme_interval["start"])-1: int(theme_interval["end"])]
#     theme = theme_ques[0]["theme"]
#     execution_time = pred_theme_ans(theme_ques, pred_out)
#     break
    
# pred_df = pd.DataFrame.from_records(pred_out)
# pred_df.head()

In [130]:
import collections
import json
import pandas as pd
import requests
import re
import string
import timeit
from ast import literal_eval

In [144]:
# NOT allowed to make changes. 
path= "/kaggle/input/apna-squad/"
# All theme prediction.
questions = json.loads(pd.read_csv(path+"train_input_question_25.csv").to_json(orient="records"))
theme_intervals = json.loads(pd.read_csv(path+"train_theme_interval_25.csv").to_json(orient="records"))
pred_out = []
theme_inf_time = {}
for theme_interval in theme_intervals:
    theme_ques = questions[int(theme_interval["start"])-1: int(theme_interval["end"])]
    theme = theme_ques[0]["theme"]
    print(theme_ques)
    # Load model fine-tuned for this theme.
    #theme_model = get_theme_model(theme)
    execution_time = timeit.timeit(lambda: pred_theme_ans(theme_ques, pred_out), number=1)
    theme_inf_time[theme_interval["theme"]] = execution_time * 1000 # in milliseconds.
    break # remove this to calculate all themes

pred_df = pd.DataFrame.from_records(pred_out)
# Write prediction to a CSV file. Teams are required to submit this csv file.
pred_df.to_csv('train_output_prediction.csv', index=False)

[{'question': "Where is Nigeria's second satellite's ground receiving station?", 'id': 1, 'theme': 'Nigeria'}, {'question': 'What Nigerian community has the worst unsustainable waste management?', 'id': 2, 'theme': 'Nigeria'}, {'question': "What was Nigeria's second satellite called?", 'id': 3, 'theme': 'Nigeria'}, {'question': 'What group was Nigeria a founding member of?', 'id': 4, 'theme': 'Nigeria'}, {'question': 'How long have international oil companies been in Nigeria?', 'id': 5, 'theme': 'Nigeria'}, {'question': 'Which tribe ran the city of Nri?', 'id': 6, 'theme': 'Nigeria'}, {'question': 'Which Western Premier was murdered in the Igbo coup?', 'id': 7, 'theme': 'Nigeria'}, {'question': 'Who failed to use the oil revenues to invest in infrastructure?', 'id': 8, 'theme': 'Nigeria'}, {'question': "What did Babangida set up to help pay Nigeria's national debt?", 'id': 9, 'theme': 'Nigeria'}, {'question': "How much of the US's oil imports come from Nigeria?", 'id': 10, 'theme': 'Ni

KeyboardInterrupt: 

In [132]:
pred_df

,question_id,paragraph_id,answers
0,1,-1,
1,2,29,west bromwich albion
2,3,30,nigcomsat - 1r
3,4,6,igbo
4,5,4,decades
5,6,6,igbo
6,7,6,abubakar tafawa balewa
7,8,-1,
8,9,-1,
9,10,4,4


In [114]:
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
        return re.sub(regex, ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    def lower(text):
    return text.lower()
  return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
    if not s: return []
    return normalize_answer(s).split()

def calc_f1(a_gold, a_pred):
    gold_toks = get_tokens(str(a_gold))
    pred_toks = get_tokens(str(a_pred))
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def calc_max_f1(predicted, ground_truths):
    max_f1 = 0
    if len(ground_truths)==0:
        max_f1= calc_f1(predicted, ground_truths)
    else:
        for ground_truth in ground_truths:
            f1 = calc_f1(predicted, ground_truth)
            max_f1 = max(max_f1, f1)
        return max_f1

In [115]:
# NOT allowed to make changes. 
out='/kaggle/working/'
# Evaluation methodology.
metrics = {}
pred = pd.read_csv(out+"train_output_prediction.csv")
truth = pd.read_csv(path+ "train_ground_truth_25.csv")
truth.paragraph_id = truth.paragraph_id.apply(literal_eval)
truth.answers = truth.answers.apply(literal_eval)
questions = pd.read_csv(path+"train_input_question_25.csv")
for idx in pred.index:
  q_id = pred["question_id"][idx]
  q_rows = questions.loc[questions['id'] == q_id].iloc[-1]
  theme = q_rows["theme"]
  predicted_paragraph = pred["paragraph_id"][idx]
  predicted_ans = pred["answers"][idx]
  
  if theme not in metrics.keys():
    metrics[theme] = {"true_positive": 0, "true_negative": 0, "total_predictions": 0, "f1_sum": 0}

  truth_row = truth.loc[truth['question_id'] == q_id].iloc[-1]
  truth_paragraph_id = [ int(i) for i in truth_row["paragraph_id"] ]
  if predicted_paragraph in truth_paragraph_id:
    # Increase TP for that theme.
    metrics[theme]["true_positive"] = metrics[theme]["true_positive"] + 1
  # -1 prediction in case there is no paragraph which can answer the query.
  if predicted_paragraph == -1 and truth_row["paragraph_id"] == []:
    # Increase TN.
    metrics[theme]["true_negative"] = metrics[theme]["true_negative"] + 1
  # Increase total predictions for that theme.
  metrics[theme]["total_predictions"] = metrics[theme]["total_predictions"] + 1
  f1 = calc_max_f1(predicted_ans, truth_row["answers"])
  
  metrics[theme]["f1_sum"] = metrics[theme]["f1_sum"] + f1

In [116]:
# NOT allowed to make changes. 

# Final score.
inf_time_threshold = 1000.0 # milliseconds.
final_para_score = 0.0
final_qa_score = 0.0
count=0

# Weight would stay hidden from teams.
#theme_weights = {"Kubernetes": 0.5, "ChatGPT": 0.4, "Football world cup": 0.1}

for theme in metrics:
  inf_time_score = 1.0
  metric = metrics[theme]
  para_score = (metric["true_positive"] + metric["true_negative"]) / metric["total_predictions"] 
  qa_score = metric["f1_sum"] / metric["total_predictions"]
  avg_inf_time = theme_inf_time[theme] / metric["total_predictions"]
  if avg_inf_time > inf_time_threshold:
    inf_time_score = inf_time_threshold / avg_inf_time
  #final_qa_score += theme_weights[theme] * inf_time_score * qa_score
  final_qa_score += inf_time_score * qa_score
  #final_para_score += theme_weights[theme] * inf_time_score * para_score
  final_para_score += inf_time_score * para_score
  count+=1

print (final_para_score/count)
print (final_qa_score/count)

0.15555555555555556
0.09481481481481481


In [118]:
count

1

In [117]:
metrics

{'Nigeria': {'true_positive': 7,
  'true_negative': 0,
  'total_predictions': 45,
  'f1_sum': 4.266666666666667}}